#Transformer Model

#### Load the dataset and preprocess the dataframe in the required format.

Technical indicators created for the environment include:

- RSI
- MACD
- Stoch_k
- OBV
- Upper_BB
- ATR_1
- ATR_2
- ATR_5
- ATR_10
- ATR_20

In [ ]:
import pandas as pd
import numpy as np
import talib as ta

class TechnicalIndicators:
    def __init__(self, data):
        self.data = data

    def add_momentum_indicators(self):
        self.data['RSI'] = ta.RSI(self.data['Close'], timeperiod=14)
        self.data['MACD'], self.data['MACD_signal'], self.data['MACD_hist'] = ta.MACD(self.data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        self.data['Stoch_k'], self.data['Stoch_d'] = ta.STOCH(self.data['High'], self.data['Low'], self.data['Close'],
                                                              fastk_period=14, slowk_period=3, slowd_period=3)

    def add_volume_indicators(self):
        self.data['OBV'] = ta.OBV(self.data['Close'], self.data['Volume'])

    def add_volatility_indicators(self):
        self.data['Upper_BB'], self.data['Middle_BB'], self.data['Lower_BB'] = ta.BBANDS(self.data['Close'], timeperiod=20)
        self.data['ATR_1'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=1)
        self.data['ATR_2'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=2)
        self.data['ATR_5'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)
        self.data['ATR_10'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=10)
        self.data['ATR_20'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=20)

    def add_trend_indicators(self):
        self.data['ADX'] = ta.ADX(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['+DI'] = ta.PLUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['-DI'] = ta.MINUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['CCI'] = ta.CCI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)

    def add_other_indicators(self):
        self.data['DLR'] = np.log(self.data['Close'] / self.data['Close'].shift(1))
        self.data['TWAP'] = self.data['Close'].expanding().mean()
        self.data['VWAP'] = (self.data['Volume'] * (self.data['High'] + self.data['Low']) / 2).cumsum() / self.data['Volume'].cumsum()

    def add_all_indicators(self):
        self.add_momentum_indicators()
        self.add_volume_indicators()
        self.add_volatility_indicators()
        self.add_trend_indicators()
        self.add_other_indicators()
        self.create_labels()
        return self.data
    def create_labels(self):
        self.data['Label'] = 0  # Default to Hold

        # Side-based labeling
        self.data.loc[self.data['side'] == 'B', 'Label'] = 1  # Buy
        self.data.loc[self.data['side'] == 'A', 'Label'] = 2  #sell

In [ ]:
data = pd.read_csv('/content/xnas-itch-20230703.tbbo.csv')

# Preprocessing to create necessary columns
data['price']=data['price']/1e9
data['bid_px_00']=data['bid_px_00']/1e9
data['ask_px_00']=data['ask_px_00']/1e9

data['Close'] = data['price']
data['Volume'] = data['size']
data['High'] = data[['bid_px_00', 'ask_px_00']].max(axis=1)
data['Low'] = data[['bid_px_00', 'ask_px_00']].min(axis=1)
data['Open'] = data['Close'].shift(1).fillna(data['Close'])


ti = TechnicalIndicators(data)
df_with_indicators = ti.add_all_indicators()
market_features_df = df_with_indicators[35:]

Checking the dataset:

In [ ]:
list(market_features_df['side']).count('N')

7896

In [ ]:
# Show all columns in pandas
pd.set_option('display.max_columns', None)

market_features_df.head(35)

In [ ]:
df_with_indicators.info()

In [ ]:
market_features_df.to_csv("market_features_df_new.csv")

### Transformer Model

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
market_features_df=pd.read_csv("/content/market_features_df_new.csv")

In [ ]:
market_features_df.columns

Index(['Unnamed: 0', 'ts_recv', 'ts_event', 'rtype', 'publisher_id',
       'instrument_id', 'action', 'side', 'depth', 'price', 'size', 'flags',
       'ts_in_delta', 'sequence', 'bid_px_00', 'ask_px_00', 'bid_sz_00',
       'ask_sz_00', 'bid_ct_00', 'ask_ct_00', 'symbol', 'Close', 'Volume',
       'High', 'Low', 'Open', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist',
       'Stoch_k', 'Stoch_d', 'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB',
       'ATR_1', 'ATR_2', 'ATR_5', 'ATR_10', 'ATR_20', 'ADX', '+DI', '-DI',
       'CCI', 'DLR', 'TWAP', 'VWAP', 'Label'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, data, seq_length, state_columns):
        self.data = data
        self.seq_length = seq_length
        self.state_columns = state_columns

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        seq = self.data[self.state_columns].iloc[idx:idx+self.seq_length].values
        label = self.data['Label'].iloc[idx+self.seq_length]
        return torch.tensor(seq, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Parameters
seq_length = 50
state_columns = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Stoch_k', 'Stoch_d',
                 'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 'ADX', '+DI', '-DI', 'CCI']
market_features_df[state_columns] = scaler.fit_transform(market_features_df[state_columns])
# Create the dataset
dataset = SequenceDataset(market_features_df, seq_length, state_columns)

# Create DataLoader
batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        src = self.embedding(src)
        src = src.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, feature_dim)
        transformer_output = self.transformer_encoder(src)
        output = self.fc(transformer_output.mean(dim=0))
        return output

# Model parameters
input_dim = len(state_columns)
model_dim = 64
num_heads = 4
num_layers = 2
output_dim = 3  # Buy, Sell, Hold

model = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim).to(device)
print(device)

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Split the data into train and test sets (80-20 split)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoader for train and test sets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Training parameters
learning_rate = 0.0005  #  Smaller learning rate
epochs = 15

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with gradient clipping
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [1/15], Loss: 0.8451
Epoch [2/15], Loss: 0.9249
Epoch [3/15], Loss: 0.7626
Epoch [4/15], Loss: 0.7905
Epoch [5/15], Loss: 0.7988
Epoch [6/15], Loss: 0.8176
Epoch [7/15], Loss: 0.7826
Epoch [8/15], Loss: 0.7004
Epoch [9/15], Loss: 0.7189
Epoch [10/15], Loss: 0.8319
Epoch [11/15], Loss: 0.6561
Epoch [12/15], Loss: 0.8328
Epoch [13/15], Loss: 0.6637
Epoch [14/15], Loss: 0.6406
Epoch [15/15], Loss: 0.7546


In [ ]:
model_path = 'transformer_model_v1.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

Model saved to transformer_model_v1.pth


In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
running_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

test_loss = running_loss / len(test_loader)
test_accuracy = correct / total * 100
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.7897, Test Accuracy: 66.78%


### TRADING BLOTTER:

#### Preprocess the data for the trading blotter:

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

INITIAL_CASH = 10_000_000  # $10 million

def preprocess_data(df):
    df['liquidity'] = df['bid_sz_00'] * df['bid_px_00'] + df['ask_sz_00'] * df['ask_px_00']
    return df

def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_vol_and_liquidity(price_df, volume_df, window_size):
    # Calculate rolling statistics
    rolling_mean_vol = price_df.pct_change().rolling(window=window_size).mean()
    rolling_std_vol = price_df.pct_change().rolling(window=window_size).std()
    rolling_mean_liq = volume_df.rolling(window=window_size).mean()
    rolling_std_liq = volume_df.rolling(window=window_size).std()

    return rolling_mean_vol, rolling_std_vol, rolling_mean_liq, rolling_std_liq

def get_percentile(current_value, mean, std):
    if std > 0:
        z_score = (current_value - mean) / std
        percentile = norm.cdf(z_score)
    else:
        percentile = 0.5  # No variation
    return percentile

def get_trade_price(base_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction):
    vol_percentile = get_percentile(current_vol, mean_vol, std_vol)
    liq_percentile = get_percentile(current_liq, mean_liq, std_liq)

    # Define price adjustment scenarios based on market conditions
    if vol_percentile >= 0.9 and liq_percentile < 0.1:
        price_adjustment_percent = np.random.uniform(-0.25, -0.15)
    elif vol_percentile <= 0.1 and liq_percentile < 0.1:
        price_adjustment_percent = np.random.uniform(-0.10, -0.05)
    elif vol_percentile >= 0.9 and liq_percentile >= 0.9:
        price_adjustment_percent = np.random.uniform(-0.05, +0.10)
    else:
        price_adjustment_percent = np.random.uniform(-0.05, +0.05)  # Default for normal conditions

    # Adjust price based on trade direction
    if trade_direction == 'BUY':
        adjusted_price = base_price * (1 - price_adjustment_percent)
    else:  # SELL
        adjusted_price = base_price * (1 + price_adjustment_percent)

    return adjusted_price


#### Create trading environment for the blotter

In [ ]:
class TradingEnvironmentwithBlotter:
    def __init__(self, data, daily_trading_limit, window_size):
        self.data = preprocess_data(data)
        self.daily_trading_limit = daily_trading_limit
        self.window_size = window_size
        self.state_columns = ['price', 'liquidity', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'Stoch_k', 'Stoch_d',
                              'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 'ADX', '+DI', '-DI', 'CCI']
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = INITIAL_CASH
        self.shares_held = 0
        self.total_shares_traded = 0
        self.cumulative_reward = 0
        self.trades = []
        self.portfolio = {'cash': self.balance, 'holdings': {ticker: 0 for ticker in self.data['symbol'].unique()}}
        self.data['RSI'] = calculate_rsi(self.data['price'])
        self.data['pct_change'] = self.data['price'].pct_change()
        self.data['rolling_mean_vol'], self.data['rolling_std_vol'], self.data['rolling_mean_liq'], self.data['rolling_std_liq'] = calculate_vol_and_liquidity(self.data['price'], self.data['liquidity'], self.window_size)

    def step(self):
        row = self.data.iloc[self.current_step]
        current_price = row['price']
        current_time = pd.to_datetime(row['ts_event'])
        current_rsi = row['RSI']
        current_vol = row['pct_change']
        current_liq = row['liquidity']
        mean_vol = row['rolling_mean_vol']
        std_vol = row['rolling_std_vol']
        mean_liq = row['rolling_mean_liq']
        std_liq = row['rolling_std_liq']

        if current_rsi < 30:  # Entry signal based on RSI
            trade_direction = 'BUY'
            trade_price = get_trade_price(current_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction)
            trade_size = (self.portfolio['cash'] * np.random.uniform(0.001, 0.005)) / trade_price
            if self.portfolio['cash'] >= trade_size * trade_price:
                self.portfolio['cash'] -= trade_size * trade_price
                self.portfolio['holdings'][row['symbol']] += trade_size
                trade_status = 'filled'
            else:
                trade_status = 'cancelled'
        elif current_rsi > 70:  # Exit signal based on RSI
            trade_direction = 'SELL'
            if self.portfolio['holdings'][row['symbol']] > 0:
                trade_size = min(self.portfolio['holdings'][row['symbol']], self.portfolio['cash']*np.random.uniform(0.001, 0.005) / current_price)
                trade_price = get_trade_price(current_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction)
                self.portfolio['cash'] += trade_size * trade_price
                self.portfolio['holdings'][row['symbol']] -= trade_size
                trade_status = 'filled'
            else:
                trade_size = 0
                trade_status = 'cancelled'
        else:
            trade_direction = 'HOLD'
            trade_size = 0
            trade_price = current_price
            trade_status = 'skipped'

        if trade_size > 0:
            expected_price = row['ask_px_00']
            actual_price = row['price']
            transaction_time = row['ts_in_delta']
            transaction_cost = self._calculate_transaction_cost(row['Volume'], 0.3, self.data['Volume'].mean())
            slippage = expected_price - actual_price
            time_penalty = 1000 * transaction_time / 1e9
            reward = - (slippage + time_penalty + transaction_cost)

            self.cumulative_reward += reward
            self.trades.append({
                'step': self.current_step,
                'timestamp': current_time,
                'action': trade_direction,
                'price': trade_price,
                'shares': trade_size,
                'symbol': row['symbol'],
                'reward': reward,
                'transaction_cost': transaction_cost,
                'slippage': slippage,
                'time_penalty': time_penalty
            })



        self.current_step += 1
        if self.current_step >= len(self.data) - 1:
            done=True
            self.current_step = 0

    def _calculate_transaction_cost(self, volume, volatility, daily_volume):
        return volatility * np.sqrt(volume / daily_volume)

    def run(self):
        self.reset()
        for _ in range(len(self.data)):
            self.step()
        return self.cumulative_reward, self.trades

    def render(self):
        print(f'Cumulative reward: {self.cumulative_reward}')
        row = self.data.iloc[self.current_step]
        print(f'Total portfolio value: {self.portfolio["cash"] + self.portfolio["holdings"][row["symbol"]]*row["Close"]}')
        # get trades in a pandas dataframe
        trades_df = pd.DataFrame(self.trades)
        # Save a csv
        trades_df.to_csv('trades_blotter.csv', index=False)
        for trade in self.trades:
            print(f"Step: {trade['step']}, Timestamp: {trade['timestamp']}, Action: {trade['action']}, Price: {trade['price']}, Shares: {trade['shares']}, Symbol: {trade['symbol']}, Reward: {trade['reward']}, Transaction Cost: {trade['transaction_cost']}, Slippage: {trade['slippage']}, Time Penalty: {trade['time_penalty']}")

#### Run the trading blotter

In [ ]:
# Filter data for the specified ticker
ticker = 'AAPL'  # Specify the ticker you want to trade
ticker_data = market_features_df[market_features_df['symbol'] == ticker]

window_size = 60
daily_trading_limit = 1000
# Create the trading environment
env = TradingEnvironmentwithBlotter(ticker_data, daily_trading_limit=1000, window_size=window_size)  # Daily trading limit of 1000 shares

# Run the environment
cumulative_reward, trades = env.run()

# Render the results
env.render()

<ipython-input-28-14d8af0093b1>:92: RuntimeWarning: invalid value encountered in sqrt
  return volatility * np.sqrt(volume / daily_volume)


Streaming output truncated to the last 5000 lines.
Step: 14939, Timestamp: 2023-07-03 13:48:36.504145419, Action: SELL, Price: 196.94519601243076, Shares: 12.55682788309265, Symbol: AAPL, Reward: -26341919.31232232, Transaction Cost: 26341919.147964317, Slippage: 0.0, Time Penalty: 0.164358
Step: 14940, Timestamp: 2023-07-03 13:48:36.504172478, Action: SELL, Price: 188.61555077594159, Shares: 17.398692109543468, Symbol: AAPL, Reward: -66125222.233492106, Transaction Cost: 66125222.069031104, Slippage: 0.0, Time Penalty: 0.164461
Step: 14941, Timestamp: 2023-07-03 13:48:36.504173973, Action: SELL, Price: 192.7548957284633, Shares: 15.901794704702903, Symbol: AAPL, Reward: -2942498.694806076, Transaction Cost: 2942498.5252460763, Slippage: 0.0, Time Penalty: 0.16956
Step: 14942, Timestamp: 2023-07-03 13:48:36.505096066, Action: SELL, Price: 186.771133338017, Shares: 27.983413008972864, Symbol: AAPL, Reward: nan, Transaction Cost: nan, Slippage: 0.009999999999990905, Time Penalty: 0.16738

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Step: 49815, Timestamp: 2023-07-03 16:29:54.454344746, Action: SELL, Price: 186.6098988119724, Shares: 0.310747461592685, Symbol: AAPL, Reward: nan, Transaction Cost: nan, Slippage: 0.0, Time Penalty: 0.170798
Step: 49816, Timestamp: 2023-07-03 16:29:54.454344746, Action: SELL, Price: 183.1189397403041, Shares: 0.31430910079589097, Symbol: AAPL, Reward: nan, Transaction Cost: nan, Slippage: 0.0, Time Penalty: 0.170798
Step: 49817, Timestamp: 2023-07-03 16:29:54.454344746, Action: SELL, Price: 195.72955192560718, Shares: 0.10920350573910671, Symbol: AAPL, Reward: nan, Transaction Cost: nan, Slippage: 0.0, Time Penalty: 0.170798
Step: 49818, Timestamp: 2023-07-03 16:29:54.474421200, Action: SELL, Price: 196.6145509170042, Shares: 0.3755465731512685, Symbol: AAPL, Reward: nan, Transaction Cost: nan, Slippage: 0.009999999999990905, Time Penalty: 0.167287
Step: 49819, Timestamp: 2023-07-03 16:29:54.474421200, Action: SELL, Price: 188.949847

In [ ]:
df=market_features_df.copy()

In [ ]:
df['timestamp']=pd.to_datetime(df['ts_recv'])

In [ ]:
df.head()

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP,timestamp
35,1688371212400103305,1688371212399937688,1,2,32,T,B,0,194.05,56,130,165617,324353,194.00,194.05,3079,56,2,1,AAPL,194.05,56,194.05,194.00,194.05,51.852848,-2.561087,-3.619556,1.058469,99.974582,99.974582,-282.0,254.713931,186.1505,117.587069,0.05,7.422578,23.240495,19.656509,12.550193,69.565924,51.717387,47.989962,0.000000,0.000000,189.649722,192.248240,2023-07-03 08:00:12.400103305
36,1688371214386057385,1688371214385893078,1,2,32,T,N,0,194.05,50,130,164307,326232,194.00,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.00,194.05,51.852848,-1.532555,-3.202156,1.669601,99.930172,99.959779,-282.0,254.718308,186.1535,117.588692,0.30,3.861289,18.652396,17.720858,11.937684,64.872152,51.762467,47.921535,166.666667,0.000000,189.768649,192.297868,2023-07-03 08:00:14.386057385
37,1688371214386063777,1688371214385899379,1,2,32,T,N,0,194.05,50,130,164398,326233,194.00,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.00,194.05,51.852848,-0.709260,-2.703577,1.994317,99.885761,99.930172,-282.0,254.721956,186.1560,117.590044,0.30,2.080645,14.981917,15.978772,11.355800,60.513649,51.683105,47.848062,83.333333,0.000000,189.881316,192.344972,2023-07-03 08:00:14.386063777
38,1688371215804852019,1688371215804687301,1,2,32,T,B,0,194.21,10,130,164718,328131,194.00,194.21,3101,29,4,1,AAPL,194.21,10,194.21,194.00,194.05,51.895447,-0.043381,-2.171538,2.128156,99.875196,99.897043,-272.0,254.737322,186.1665,117.595678,0.21,1.145322,12.027533,14.401895,10.798510,56.466467,51.623439,47.792823,79.268293,0.000824,189.992308,192.353879,2023-07-03 08:00:15.804852019
39,1688371216978631317,1688371216978466819,1,2,470,T,A,0,114.57,43,130,164498,329439,114.57,114.76,43,27,1,1,AMD,114.57,43,114.76,114.57,194.21,35.201020,-5.874236,-2.912077,-2.962159,83.062571,94.274509,-315.0,257.373490,182.1950,107.016510,79.64,40.392661,25.550027,20.925705,14.240584,54.540006,35.082367,64.436445,-166.666667,-0.527754,188.106750,190.699287,2023-07-03 08:00:16.978631317
